In [187]:
import folium
import pandas as pd
import geopandas as gpd

In [188]:
import pandas
url = "http://localhost:5001/generate"
data = pandas.read_json(url)

In [189]:
data

,id_external_object,id_instance_object,lat,lon,type_object
0,Brunehaut,1001,48.852748,2.307051,personnage
1,"Clotaire Ier, roi des Francs de Soissons résid...",1002,48.818192,2.375219,evenement
2,Pathir,1003,48.820121,2.308830,personnage
3,Clotaire II,1004,48.851636,2.400777,roi
4,Amelius est évêque de Paris.,1005,48.878620,2.341129,evenement
...,...,...,...,...,...
95,Eusebius,1096,48.876665,2.282682,personnage
96,Chilpéric revient à Paris.,1097,48.818386,2.397043,evenement
97,Février 583 : Première crue importante de la S...,1098,48.869746,2.284533,evenement
98,"Mort de [saint] Germain, évêque de Paris ; il ...",1099,48.836517,2.387682,evenement


In [190]:
m = folium.Map(location=[48.853,2.35], tiles="OpenStreetMap", zoom_start=12)

In [194]:
colors = {
    'roi':'red',
    'evenement':'blue',
    'personnage':'purple'
}

In [192]:
for index, row in data.iterrows():
    folium.Marker([row['lat'],row['lon']], popup='<i>'+str(row['id_external_object'])+'</i>', tooltip=str(row['id_external_object']), icon=folium.Icon()).add_to(m)

In [193]:
m